# Utilizzo del User-Based Collaborative Filtering

In [1]:
# import delle librerie necessarie
import numpy as np
import pandas as pd

In [2]:
# funzione per trovare la similarità tra due utenti: Coefficiente di Pearson
def weight_factor(x, y): 
    t1, t2, t3 = 0, 0, 0 
    for i, j in zip(x, y):
        t1+=i*j
        t2+=i*i
        t3+=j*j
    return t1/(np.sqrt(t2) * np.sqrt(t3))

In [3]:
# lettura dei rating normalizzati dal relativo CSV
normalized_ratings_df = pd.read_csv('./data/normalized_ratings.csv')

# lettura dei rating non normalizzati dal relativo CSV
ratings_df = pd.read_csv('./data/jester_jokes_ratings.csv')

normalized_ratings_df.head(2)

,user_id,number_of_jokes_rated,joke_1,joke_2,joke_3,joke_4,joke_5,joke_6,joke_7,joke_8,...,joke_91,joke_92,joke_93,joke_94,joke_95,joke_96,joke_97,joke_98,joke_99,100
0,1,74,-4.388108,12.221892,-6.228108,-4.728108,-4.088108,-5.068108,-6.418108,7.601892,...,6.251892,0.0000,0.0000,0.0000,0.0000,0.0000,-2.198108,0.0000,0.0000,0.0000
1,2,100,1.333700,-3.036300,3.613700,1.623700,-5.126300,-12.406300,-3.476300,-8.086300,...,0.073700,-7.6963,-3.0363,5.1137,-2.9363,-4.8863,0.313700,-2.4063,-7.0663,-1.6763


## Separazione dei rating normalizzati

Si separano i rating noramlizzati in due parti: 
1. una parte contenente tutti gli utenti che hanno valutato tutte e 100 le barzellette;
2. una parte contenente tutti gli utenti che non hanno valutato tutte le barzellette.

Si seleziona poi l'utente target (al quale si vuole fare una raccomandazione) dalla matrice sparsa e si cercano i vicini di quell'utente nella la matrice non sparsa.

In [4]:
# utenti che hanno valutato tutte e 100 le barzellette
complete_ratings = normalized_ratings_df[normalized_ratings_df['number_of_jokes_rated'] == 100]
print('totale utenti che hanno valutato tutte le barzellette: ', len(complete_ratings))

# utenti che non hanno valutato tutte le barzellette
sparse_ratings = normalized_ratings_df[normalized_ratings_df['number_of_jokes_rated'] != 100]
print('totale utenti che non hanno valutato tutte le barzellette: ', len(sparse_ratings))

totale utenti che hanno valutato tutte le barzellette:  7200
totale utenti che non hanno valutato tutte le barzellette:  17783


In [5]:
# selezione dell'utente target in maniera random
n = 1000
active_user_id = sparse_ratings.iloc[n, 0]
print("id dell'utente target: {}".format(str(active_user_id)))

id dell'utente target: 1352


In [6]:
print('valutazioni date da utente con id {} per le 100 barzellette'.format(str(active_user_id)))
active_user = sparse_ratings[sparse_ratings['user_id'] == active_user_id]
active_user_rating = active_user.iloc[:, 2:]
active_user_rating

valutazioni date da utente con id 1352 per le 100 barzellette


,joke_1,joke_2,joke_3,joke_4,joke_5,joke_6,joke_7,joke_8,joke_9,joke_10,...,joke_91,joke_92,joke_93,joke_94,joke_95,joke_96,joke_97,joke_98,joke_99,100
1351,7.871096,-1.888904,-2.568904,-1.118904,2.821096,-4.078904,-0.188904,1.891096,-2.568904,3.931096,...,0.0,-3.448904,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [7]:
# vengono messi i rating dell'utente target in una lista
active_user_rating_list = active_user_rating.values.ravel()

# si calcola la similarità tra l'utente target e gli utenti della matrice non sparsa
similarity = np.array([(complete_ratings.iloc[i, 0],\
             weight_factor(active_user_rating_list, complete_ratings.iloc[i, 2:]))\
             for i in range(complete_ratings.shape[0])])

In [8]:
# si ordinano i neighbor più simili in ordine decrescente
ind = np.argsort( similarity[:,1] )
similarity = similarity[ind]
similarity

array([[ 7.33800000e+03, -2.87912874e-01],
       [ 1.66100000e+03, -2.87005484e-01],
       [ 1.41410000e+04, -2.70326403e-01],
       ...,
       [ 1.92170000e+04,  3.99469178e-01],
       [ 1.06150000e+04,  4.06355890e-01],
       [ 7.49200000e+03,  4.59458050e-01]])

## Si prendono i primi 30 neighbor più simili all'utente target che rispettano una certa soglia (similarità maggiore di 0,1)

In [10]:
neighbors = similarity[similarity[:,1] > 0.1]
print('Si hanno {} potenziali neighbor! Se ne selezionano 30 tra questi'.format(len(neighbors)))

Si hanno 2323 potenziali neighbor! Se ne selezionano 30 tra questi


In [13]:
# con replace = False si è sicuri che non un neighbor non viene selezionato più volte
index_30_neighbor = np.random.choice(range(len(neighbors)), 30, replace=False)
selected_neighbors = neighbors[index_30_neighbor]
selected_neighbors

array([[1.94580000e+04, 1.64382565e-01],
       [1.88900000e+04, 1.52063382e-01],
       [1.42700000e+03, 1.25800525e-01],
       [7.71100000e+03, 1.94052226e-01],
       [6.11900000e+03, 1.09001846e-01],
       [2.20790000e+04, 1.66095591e-01],
       [3.81800000e+03, 2.05102031e-01],
       [2.45520000e+04, 1.07112632e-01],
       [5.32400000e+03, 2.05523407e-01],
       [6.20500000e+03, 2.08801380e-01],
       [2.33300000e+04, 1.17581778e-01],
       [2.34780000e+04, 1.68556483e-01],
       [6.86000000e+02, 1.33127964e-01],
       [2.20800000e+03, 1.39041383e-01],
       [3.63000000e+03, 2.65067408e-01],
       [2.38740000e+04, 1.92101443e-01],
       [2.01550000e+04, 1.26367481e-01],
       [2.48000000e+03, 1.16942649e-01],
       [1.70170000e+04, 1.83085390e-01],
       [2.24910000e+04, 1.01970684e-01],
       [1.94360000e+04, 1.08773329e-01],
       [7.76700000e+03, 1.27624227e-01],
       [2.11420000e+04, 1.57645338e-01],
       [1.20470000e+04, 2.35953051e-01],
       [2.195100

## Assegnazione degli score per le barzellette non ancora valutate dall'utente target 

In [14]:
recommendation_columns = [column for column in active_user_rating.columns if active_user_rating[column].values[0] == 0]

In [15]:
# si calcola la media delle valutazioni dello user target in modo 
# da avere una predizione più precisa (formula vista nel corso)
active_user_raw_ratings = ratings_df[ratings_df['user_id'] == active_user_id].iloc[:, 2:]
active_user_mean_rating = np.mean(active_user_raw_ratings.drop(recommendation_columns, axis = 1).values)
active_user_mean_rating

0.6289041095890411

In [17]:
# user id dei neighbor
neighbour_user_id = selected_neighbors[:, 0]

# similarità dei neighbor con lo user target
neighbour_user_similarity = selected_neighbors[:, 1]

print('user id dei neighbor: ', neighbour_user_id, '\n\n')
print('similarità tra utente e neighbor: ', neighbour_user_similarity)

user id dei neighbor:  [19458. 18890.  1427.  7711.  6119. 22079.  3818. 24552.  5324.  6205.
 23330. 23478.   686.  2208.  3630. 23874. 20155.  2480. 17017. 22491.
 19436.  7767. 21142. 12047. 21951. 22898.  5818.  8104.  9241.  6952.] 


similarità tra utente e neighbor:  [0.16438256 0.15206338 0.12580053 0.19405223 0.10900185 0.16609559
 0.20510203 0.10711263 0.20552341 0.20880138 0.11758178 0.16855648
 0.13312796 0.13904138 0.26506741 0.19210144 0.12636748 0.11694265
 0.18308539 0.10197068 0.10877333 0.12762423 0.15764534 0.23595305
 0.17537201 0.15210636 0.14446782 0.24513896 0.14885038 0.10830314]


In [18]:
# si prendono i rating dei neighbor dell'utente target
neighbours_df = complete_ratings[complete_ratings['user_id'].isin(neighbour_user_id)]
len(neighbours_df)

30

In [19]:
# si selezionano solo le barzellette che si volgiono raccomandare all'utente (quelle che non ha valutato)
print('Si suggerirà una tra le {} barzellette \n\n'.format(len(recommendation_columns)))

neighbours_df = neighbours_df[recommendation_columns]
neighbours_df.head()

Si suggerirà una tra le 27 barzellette 




,joke_71,joke_72,joke_73,joke_74,joke_75,joke_76,joke_77,joke_78,joke_80,joke_82,...,joke_90,joke_91,joke_93,joke_94,joke_95,joke_96,joke_97,joke_98,joke_99,100
685,5.3295,-4.3305,-3.5505,-4.1905,-3.6505,3.0495,-4.7705,-3.6505,-6.0305,-4.8705,...,11.2495,11.6895,-6.5605,-4.6205,-2.7805,-3.3605,-4.3805,0.2295,-1.9005,-2.5405
1426,10.6031,4.2431,1.7231,3.8531,-3.2269,3.0331,-5.2169,10.2631,7.7931,4.3931,...,3.7131,5.5531,3.5631,-1.5769,1.2831,4.0031,2.3531,2.7431,0.3131,5.8031
2207,1.9397,4.5597,-2.8203,-1.2603,4.7597,4.3697,-7.2303,-0.0503,-0.8303,5.0497,...,-1.7503,4.7097,0.1497,4.3697,4.5097,3.4497,5.4897,1.3597,0.6297,5.8697
2479,2.6556,2.6556,-8.3644,0.3256,-7.7744,3.3356,3.3356,3.3356,-1.9044,5.5256,...,6.8856,3.0456,6.5456,-8.8444,2.5656,-8.4144,-8.4144,7.1256,6.6856,-9.1344
3629,-4.5946,4.9654,-3.2846,3.5154,3.6554,3.3654,4.4354,4.3354,0.7954,1.8654,...,2.2954,4.8254,1.9054,3.8554,-7.0246,2.7354,1.3754,-0.6646,-0.1746,1.1354


In [20]:
item_id = recommendation_columns[0]
print('predizione della valutazione della prima barzelletta', item_id)

def score_user_item(item_id, neighbours_df,neighbour_user_similarity, active_user_mean_rating ):
    item_rating = neighbours_df[item_id]
    t1, t2 = 0, 0
    for similarity, norm_rating in zip(neighbour_user_similarity, item_rating):
        t1+= norm_rating * similarity
        t2+= similarity
    score = (t1 + active_user_mean_rating)/t2
    return score

# predizione relativa alla prima barzelletta
score_user_item(item_id, neighbours_df,neighbour_user_similarity, active_user_mean_rating )
        

predizione della valutazione della prima barzelletta joke_71


-1.1901602640229152

In [21]:
# raccomandazione della barzelletta con la predizione più alta
top_score = -np.inf
joke_to_suggest = ''

for column in neighbours_df.columns:
    score =score_user_item(column, neighbours_df,neighbour_user_similarity, active_user_mean_rating)
    if score > top_score:
        top_score = score
        joke_to_suggest = column
print('il valore più alto è', top_score)
print('la barzelletta da raccomandare è: ', joke_to_suggest)    

il valore più alto è 3.0727721508315042
la barzelletta da raccomandare è:  joke_89
